In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import os
import time
from datetime import datetime, timedelta
from sklearn import preprocessing
import pdb
import matplotlib.pyplot as plt
train = pd.read_csv('./farming.csv')
test = pd.read_csv('./product_market.csv')
valid_357 = pd.read_csv('./3_5_7_min_withhead.csv')
# train_selectby_test = train.sort_values(by=['市场名称映射值','农产品名称映射值','数据发布时间'])
mappingMarket_test = set(test['市场名称映射值'])
mappingName_test = set(test['农产品名称映射值'])
train_selectby_Market_test = train[train['市场名称映射值'].isin(mappingMarket_test)]
train_selectby_Name_test = train_selectby_Market_test[train_selectby_Market_test['农产品名称映射值'].isin(mappingName_test)]
train_selectby_test = train_selectby_Name_test
# train_selectby_test.head()

In [3]:
len(train_selectby_Market_test),len(train_selectby_test)

(718787, 607511)

In [4]:
train_selectby_test_backup = train_selectby_test #选取的测试集的备份,同时过滤掉颜色，规格，单位，区域，入库
train_selectby_test.drop('区域', axis=1, inplace=True)
train_selectby_test.drop('规格', axis=1, inplace=True)
train_selectby_test.drop('颜色', axis=1, inplace=True)
train_selectby_test.drop('单位', axis=1, inplace=True)
train_selectby_test.drop('数据入库时间', axis=1, inplace=True)
filter_train_selectby_test = train_selectby_test #该数据已经过滤掉颜色，规格，单位，区域，入库等时间影响
# filter_train_selectby_test
#该数据保存的是5，6月的价格数据
# filter_train_selectby_test_leasttwomonth = filter_train_selectby_test[filter_train_selectby_test['数据发布时间']>'2016-04-31']
filter_train_selectby_test_leasttwomonth = filter_train_selectby_test #所有数据
#newdata_orderbyMNT 记录最近两个月5/6月，按照market,name,time排序后的结果
newdata_orderbyMNT = filter_train_selectby_test_leasttwomonth.sort_index(by=['市场名称映射值','农产品名称映射值','数据发布时间'])
# newdata_orderbyMNT
valid_357_sorted = valid_357.sort_index(by=['name','market'])

D:\File after install\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
D:\File after install\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
D:\File after install\Anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\File after install\Anaconda\lib\site-packages\ipykernel\__main__

In [5]:
#simple

# given a series and alpha, return series of smoothed points
def exponential_smoothing(series, alpha):
#     result = [series[0]] # first value is same as series
    result = [sum(series)/len(series)] #avg for first value
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

#double

# given a series and alpha, return series of smoothed points
def exponential_smoothing(series, alpha):
#     result = [series[0]] # first value is same as series
    result = [sum(series)/len(series)] #avg for first value
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

#triple
def initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def triple_exponential_smoothing(series, slen, alpha, beta, gamma, n_preds):
    result = []
    seasonals = initial_seasonal_components(series, slen)
    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series[0]
            trend = initial_trend(series, slen)
            result.append(series[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%slen])
        else:
            val = series[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%slen]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%slen] = gamma*(val-smooth) + (1-gamma)*seasonals[i%slen]
            result.append(smooth+trend+seasonals[i%slen])
    return result

In [93]:
#针对特定模块进行模型预测
#根据最近几天预测当天的结果
pre_name,pre_type,pre_market = None,None,None
curve_line = pd.read_csv('C:/Users/Aaron_Huang/Desktop/CCF_2016/pltsave/pltsave/curve_line.csv',header=None)
finalresult = []
totalerror = 0
resultdate = ['2016-07-01','2016-07-02','2016-07-03','2016-07-04','2016-07-05','2016-07-06','2016-07-07','2016-07-08','2016-07-09','2016-07-10',
             '2016-07-11','2016-07-12','2016-07-13','2016-07-14','2016-07-15','2016-07-16','2016-07-17','2016-07-18','2016-07-19','2016-07-20',
             '2016-07-21','2016-07-22','2016-07-23','2016-07-24','2016-07-25','2016-07-26','2016-07-27','2016-07-28','2016-07-29',
             '2016-07-30','2016-07-31']
count = 0
for data in curve_line[0].values:
#         pdb.set_trace()
        data = str(data)
#         pdb.set_trace()
        data_market = data.split('_')[0]
        data_type = data.split('_')[2]
        data_name = data.split('_')[1]
        count += 1
        tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
        tmpselect2 = tmpselect1[tmpselect1['农产品类别']==data_type]
        finalselect = tmpselect2[tmpselect2['市场名称映射值']==data_market]
        pricelist = [value for value in finalselect[-10:]['平均交易价格'].values]
        tmptype = finalselect[-1:]['农产品类别'].values[0]
        for date in resultdate:
#             tmpprice = sum([everyday_rate[i]*pricelist[i] for i in range(len(everyday_rate))])
#             tmpprice = triple_exponential_smoothing(pricelist, 7, 0.8, 0.03, 0.95, 31)
            tmpprice = exponential_smoothing(pricelist,0.95)
            tmperror = 0
            for index in range(len(pricelist)-1):
                tmperror += abs(pricelist[index+1]-tmpprice[index])/pricelist[index+1]
            totalerror += tmperror
            tmpresult = [data_market,tmptype,data_name,date,tmpprice[-1]]
            finalresult.append(tmpresult)
            del pricelist[0]
            pricelist.append(tmpprice[-1])
#         pdb.set_trace()
totalerror

424.1490871664775

In [109]:
# 特定模块和其他融合获得最终答案
pre_name,pre_type,pre_market = None,None,None
curve_line = pd.read_csv('C:/Users/Aaron_Huang/Desktop/CCF_2016/pltsave/pltsave/curve_line.csv',header=None)
plt_line = pd.read_csv('C:/Users/Aaron_Huang/Desktop/CCF_2016/pltsave/pltsave/pltsave.csv',header=None)
finalresult = []
totalerror = 0
resultdate = ['2016-07-01','2016-07-02','2016-07-03','2016-07-04','2016-07-05','2016-07-06','2016-07-07','2016-07-08','2016-07-09','2016-07-10',
             '2016-07-11','2016-07-12','2016-07-13','2016-07-14','2016-07-15','2016-07-16','2016-07-17','2016-07-18','2016-07-19','2016-07-20',
             '2016-07-21','2016-07-22','2016-07-23','2016-07-24','2016-07-25','2016-07-26','2016-07-27','2016-07-28','2016-07-29',
             '2016-07-30','2016-07-31']
count = 0
curve_list = []
plt_list = []
for data in plt_line[0].values:
#         pdb.set_trace()
        data = str(data)
#         pdb.set_trace()
        data_market = data.split('_')[0]
        data_type = data.split('_')[2]
        data_name = data.split('_')[1]
        plt_list.append([data_market,data_name,data_type])
        tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
        tmpselect2 = tmpselect1[tmpselect1['农产品类别']==data_type]
        finalselect = tmpselect2[tmpselect2['市场名称映射值']==data_market]
        pricelist = [value for value in finalselect[-3:]['平均交易价格'].values]
        tmptype = finalselect[-1:]['农产品类别'].values[0]
        for date in resultdate:
#             tmpprice = sum([everyday_rate[i]*pricelist[i] for i in range(len(everyday_rate))])
#             tmpprice = triple_exponential_smoothing(pricelist, 7, 0.8, 0.03, 0.95, 31)
            tmpprice = exponential_smoothing(pricelist,0.92)
            tmperror = 0
            for index in range(len(pricelist)-1):
                tmperror += abs(pricelist[index+1]-tmpprice[index])/pricelist[index+1]
            totalerror += tmperror
            tmpresult = [data_market,tmptype,data_name,date,tmpprice[-1]]
            finalresult.append(tmpresult)
            del pricelist[0]
            pricelist.append(tmpprice[-1])
print(totalerror)
for data in curve_line[0].values:
#         pdb.set_trace()
        data = str(data)
#         pdb.set_trace()
        data_market = data.split('_')[0]
        data_type = data.split('_')[2]
        data_name = data.split('_')[1]
        curve_list.append([data_market,data_name,data_type])
        count += 1
        tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
        tmpselect2 = tmpselect1[tmpselect1['农产品类别']==data_type]
        finalselect = tmpselect2[tmpselect2['市场名称映射值']==data_market]
        pricelist = [value for value in finalselect[-3:]['平均交易价格'].values]
        tmptype = finalselect[-1:]['农产品类别'].values[0]
        for date in resultdate:
#             tmpprice = sum([everyday_rate[i]*pricelist[i] for i in range(len(everyday_rate))])
#             tmpprice = triple_exponential_smoothing(pricelist, 7, 0.8, 0.03, 0.95, 31)
            tmpprice = exponential_smoothing(pricelist,0.92)
            tmperror = 0
            for index in range(len(pricelist)-1):
                tmperror += abs(pricelist[index+1]-tmpprice[index])/pricelist[index+1]
            totalerror += tmperror
            tmpresult = [data_market,tmptype,data_name,date,tmpprice[-1]]
            finalresult.append(tmpresult)
            del pricelist[0]
            pricelist.append(tmpprice[-1])
print(totalerror)

for data_357 in valid_357_sorted.values:
#         pdb.set_trace() 
        data_market = data_357[0]
        data_type = data_357[1]
        data_name = data_357[2]
        if pre_name == data_name and pre_market == data_market and pre_type==data_type:
            continue
        elif [data_market,data_name,data_type] in curve_list:
            continue
        elif [data_market,data_name,data_type] in plt_list:
            continue
        else:
            try:
                count += 1
                tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
                tmpselect2 = tmpselect1[tmpselect1['农产品类别']==data_type]
                finalselect = tmpselect2[tmpselect2['市场名称映射值']==data_market]
    #             pdb.set_trace()
                pricelist = [value for value in finalselect[-3:]['平均交易价格'].values]
                tmptype = finalselect[-1:]['农产品类别'].values[0]
                for date in resultdate:
                    tmpresult = [data_market,tmptype,data_name,date,pricelist[-1]]
                    finalresult.append(tmpresult)
            except Exception as e:
                print(traceback.print_exc())
#                 pdb.set_trace()
            pre_name = data_name
            pre_type = data_type
            pre_market = data_market


64.1816262997
90.2954054363


In [110]:
len(finalresult)

43586

In [ ]:
#根据最近几天预测当天的结果
import traceback
pre_name,pre_type,pre_market = None,None,None
everyday_rate = [0.5,0.5] #每一天的比重
finalresult = []
resultdate = ['2016-07-01','2016-07-02','2016-07-03','2016-07-04','2016-07-05','2016-07-06','2016-07-07','2016-07-08','2016-07-09','2016-07-10',
             '2016-07-11','2016-07-12','2016-07-13','2016-07-14','2016-07-15','2016-07-16','2016-07-17','2016-07-18','2016-07-19','2016-07-20',
             '2016-07-21','2016-07-22','2016-07-23','2016-07-24','2016-07-25','2016-07-26','2016-07-27','2016-07-28','2016-07-29',
             '2016-07-30','2016-07-31']
count = 0
for data_357 in valid_357_sorted.values:
#         pdb.set_trace() 
        data_market = data_357[0]
        data_type = data_357[1]
        data_name = data_357[2]
        if pre_name == data_name and pre_market == data_market and pre_type==data_type:
            continue
        else:
            try:
                count += 1
                tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
                tmpselect2 = tmpselect1[tmpselect1['农产品类别']==data_type]
                finalselect = tmpselect2[tmpselect2['市场名称映射值']==data_market]
    #             pdb.set_trace()
                pricelist = [value for value in finalselect[-30:]['平均交易价格'].values]
                tmptype = finalselect[-1:]['农产品类别'].values[0]
                for date in resultdate:
#                     tmpprice = sum([everyday_rate[i]*pricelist[i] for i in range(len(everyday_rate))])
                    tmpprice = triple_exponential_smoothing(pricelist, 7, 0.8, 0.03, 0.95, 31)
                    tmpresult = [data_market,tmptype,data_name,date,tmpprice[-1]]
                    finalresult.append(tmpresult)
                    del pricelist[0]
                    pricelist.append(tmpprice[-1])
            except Exception as e:
                print(traceback.print_exc())
#                 pdb.set_trace()
            pre_name = data_name
            pre_type = data_type
            pre_market = data_market
          

In [111]:
import csv
with open('multicurve_plt_3_0.92_0.92.csv', 'w',newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
#     for row in finalresult:
#         spamwriter.writerow(row)
    spamwriter.writerows(finalresult)

In [84]:
newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']=='E312F7D3A019D0A14847B38E09A15EAF']

,农产品市场所在省份,市场名称映射值,农产品类别,农产品名称映射值,最低交易价格,平均交易价格,最高交易价格,数据发布时间
652256,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.25,25.5,2013-06-10
652257,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.25,25.5,2013-06-11
652258,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.25,25.5,2013-06-13
652259,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.25,25.5,2013-06-16
652346,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.50,26.0,2013-06-17
652347,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.50,26.0,2013-06-18
652348,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.50,26.0,2013-06-19
652349,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.50,26.0,2013-06-20
652350,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.50,26.0,2013-06-21
652351,北京,4A0234139A014A1DC38F4A5243106323,畜禽蛋品,E312F7D3A019D0A14847B38E09A15EAF,25.0,25.50,26.0,2013-06-22


In [33]:
#保存价格没有变化的数据
nochangeList = []
sixList = []
pre_name,pre_type,pre_market = None,None,None
for data_357 in valid_357_sorted.values:
#         pdb.set_trace() 
    data_market = data_357[0]
    data_type = data_357[1]
    data_name = data_357[2]
    if pre_name == data_name and pre_market == data_market and pre_type==data_type:
        continue
    else:
        tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
        tmpselect2 = tmpselect1[tmpselect1['农产品类别']==data_type]
        finalselect = tmpselect2[tmpselect2['市场名称映射值']==data_market]
#         finalselect = finalselect[finalselect['数据发布时间']>'2016-04-31']
        finalselect = finalselect[-60:]
        if len(finalselect)!=0 and finalselect['平均交易价格'].values[0] == finalselect['平均交易价格'].values[-1]:
            sixList.append([data_market,data_name])
#             continue
#             pdb.set_trace()
        pre_name = data_name
        pre_market = data_market
        pre_type = data_type

In [52]:
#保存近两个月都没有变化的列表
# MayJuneList = sixList
# NoChangeList[0]
# len(MayJuneList)
# NoChangeList = sixList
# len(MayJuneList),len(NoChangeList),len(sixList)
# OtherList
# len(OtherList)
# NoChangeList

[['2D9DC863133DC9BC81E8A059E714F381', '002DFF1A88976EB3BACAD2269B4E516F'],
 ['2D9DC863133DC9BC81E8A059E714F381', '01243354E6804B0D22A9854DC35DDB8E'],
 ['2D9DC863133DC9BC81E8A059E714F381', '01DB36FB8D06075CB38C6510C86FD045'],
 ['2D9DC863133DC9BC81E8A059E714F381', '02173804DA69FA1BBAEF25C700992159'],
 ['2D9DC863133DC9BC81E8A059E714F381', '024AD3C1BBA398BA0EC835043A385E20'],
 ['2D9DC863133DC9BC81E8A059E714F381', '04064149AB86E14B0B0E4409DDA21619'],
 ['4A0234139A014A1DC38F4A5243106323', '0455D6149F450C26D2F79F829272B44A'],
 ['CFF22955C0CF6671196860C836594E7D', '058834C58953C678A7F4152C89B3CB36'],
 ['2D9DC863133DC9BC81E8A059E714F381', '05BCF5ED888AAD32BF66D9DE3BE83EA0'],
 ['2D9DC863133DC9BC81E8A059E714F381', '05D9A4C487B14C23CF522C99B6671B11'],
 ['2D9DC863133DC9BC81E8A059E714F381', '05E7C2E75334E9F9289049037785068D'],
 ['2D9DC863133DC9BC81E8A059E714F381', '06B972283A6F09BEF7DEBE7C349BC663'],
 ['F84FFE619392149018384D16BE6FF525', '07943F9A8C956B6EF795FFAF8FD1FDA2'],
 ['4A0234139A014A1DC38F4A

In [55]:
isFirst = 0
for data in MayJuneList:
    data_market = data[0]
    data_name = data[1]
    tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
    if isFirst==0:
        finalselect = tmpselect1[tmpselect1['市场名称映射值']==data_market][1:]
    else:
        finalselect = pd.concat([finalselect,tmpselect1[tmpselect1['市场名称映射值']==data_market][1:]])
#     pdb.set_trace()
    isFirst = 1
finalselect.to_csv('./split_dataset/MayJuneList.csv',index=False)

In [58]:
#画图
pre_name,pre_tyoe,pre_market = None,None,None
for data_357 in valid_357_sorted.values:
#         pdb.set_trace() 
        data_market = data_357[0]
        data_type = data_357[1]
        data_name = data_357[2]
        if (pre_name == data_name and pre_market == data_market and pre_type==data_type):
            continue
        elif [data_market,data_name] in NOCHANGELIST:
            continue
        elif [data_market,data_name] in MayJuneList:
            continue
        else:
            tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
            tmpselect2 = tmpselect1[tmpselect1['农产品类别']==data_type]
            finalselect = tmpselect2[tmpselect2['市场名称映射值']==data_market]
#             print(finalselect)
            finalselect = finalselect[-100:]
            fig = plt.figure(figsize=(8,6))
            finalselect['数据发布时间'] = finalselect['数据发布时间'].apply(lambda x:int(x.replace('-','')))
            plt.plot(finalselect['数据发布时间'],finalselect['平均交易价格'],'ro',lw=2.0, linestyle="-")
            plt.xlabel('Time')
            plt.ylabel('Price')
#             plt.show()
#             pdb.set_trace()
            fig.savefig('./pltsave/'+data_name+'_'+data_type+'.jpg')
            plt.close()
            pre_name = data_name
            pre_market = data_market
            pre_type==data_type
#             pdb.set_trace()

D:\File after install\Anaconda\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

In [59]:
count = 0
tmpList = MayJuneList
for data in NoChangeList:
    data_market = data[0]
    data_name = data[1]
    if [data_market,data_name] in OtherList:
        count += 1
#         NoChangeList.remove([data_market,data_name])
#         tmpList.append([data_market,data_name])
print(count)

0


In [43]:
len(tmpList)

990

In [48]:
# NOCHANGELIST , MayJuneList
pre_name,pre_type,pre_market = None,None,None
count,Nocount,count56,dupcount = 0,0,0,0
other_data = []
for data_357 in valid_357_sorted.values:
#         pdb.set_trace() 
    data_market = data_357[0]
    data_type = data_357[1]
    data_name = data_357[2]
    if pre_name == data_name and pre_market == data_market and pre_type==data_type:
        dupcount += 1
        continue
    elif [data_market,data_name] in NoChangeList:
#         pdb.set_trace()
        Nocount += 1
        pre_name = data_name
        pre_market = data_market
        pre_type = data_type
        continue
    elif [data_market,data_name] in tmpList:
        pre_name = data_name
        pre_market = data_market
        pre_type = data_type
        count56 += 1
        continue
    else:
        count += 1
        other_data.append([data_market,data_name])
#             tmpselect1 = newdata_orderbyMNT[newdata_orderbyMNT['农产品名称映射值']==data_name]
#             tmpselect2 = tmpselect1[tmpselect1['农产品类别']==data_type]
#             finalselect = tmpselect2[tmpselect2['市场名称映射值']==data_market]
        pre_name = data_name
        pre_market = data_market
        pre_type = data_type
print(count,Nocount,count56,dupcount)
print(len(other_data))

100 347 959 42179
100


In [112]:
# finalselect